# This file is created to Change the baseline dataset into a dupedataset 

In [1]:
import os
import pandas as pd

Address_df = pd.read_csv('Data_source/Sample_35_train/train_data/Address.csv')  
ContactPoint_df = pd.read_csv('Data_source/Sample_35_train/train_data/ContactPoint.csv')
HealthcareOrganization_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcareOrganization.csv')
HealthcarePersonnel_df = pd.read_csv('Data_source/Sample_35_train/train_data/HealthcarePersonnel.csv')
Person_df = pd.read_csv('Data_source/Sample_35_train/train_data/Person.csv')
ServiceDepartment_df = pd.read_csv('Data_source/Sample_35_train/train_data/ServiceDepartment.csv')

# Specify the encoding to handle the decoding error
golden_record = pd.read_csv('train_golden_standard_duplicates.csv', encoding='latin1')

In [2]:
golden_record

,original_id,duplicate_id,entity_type,variation_type,field_name,original_value,varied_value
0,4e48ce3c-38a1-408d-b06f-0d4ac4092209,6b46dba4-5830-5ee3-ad90-707c1b1d29a6,Address,country_expansion,country,NL,Netherlands
1,9cdeb3e6-0870-415c-afcd-81b5d24bace4,389078b5-7572-5a7f-8f19-108e0ac2ede0,Address,postal_format,postalCode,64161,64161
2,68cce369-339b-4ce8-8bb3-66b225a3fcce,ccc37f61-f924-5456-99f9-041bbba8201c,Address,country_expansion,country,AT,Austria
3,d4e3fd71-e861-43fd-bca1-248409aaae6d,1eac3582-b4d2-5d52-b330-b354033b3d21,Address,country_expansion,country,AT,Austria
4,3464a4d6-7f6e-45a6-80b3-5a929e0394e7,dbc8a716-7908-50f6-b7ce-c93773364e1f,Address,city_typo,city,Zeijerveen,Zeijerven
...,...,...,...,...,...,...,...
2223,b6c843e4-1325-4452-ac29-c1bfb0791b94,b4f959ff-b472-5077-804b-fda26f7438d2,ContactPoint,email_typo,email,Carter.Cardiovascular@dept.healthcare.org,Cartre.Cardiovascular@dept.healthcare.org
2224,9e996246-f4d3-435c-81d7-4a3ce30481a5,eec45c8b-babe-53b0-9bdb-a033fd685021,ContactPoint,email_typo,email,ValenciaThomas.Infectious@dept.healthcare.org,ValenciaThomasg.Infectious@dept.healthcare.org
2225,7e48a1c8-ef4d-478a-b195-c85b1a78dfc0,0a2e9c6c-3e5c-53ed-b311-e0a18913cd01,ContactPoint,email_domain_change,email,Lam.Infectious@dept.healthcare.org,Lam.Infectious@dept.healthcare.nl
2226,2360db82-1a14-4f91-8a34-065b2b058a0e,0330be73-3337-54b1-8340-f3b87935bf53,ContactPoint,email_domain_change,email,Nash.Pharmacy@dept.healthcare.org,Nash.Pharmacy@dept.healthcare.nl


In [3]:
# Dictionary to store all dataframes by entity type
entity_dataframes = {}

# Get a static list of variable names first to avoid the iteration error
variable_names = list(locals().keys())

# Populate the dictionary with your existing dataframes
for var_name in variable_names:
    if var_name.endswith('_df'):
        entity_type = var_name.replace('_df', '')
        entity_dataframes[entity_type] = locals()[var_name]

# Iterate through each row in the golden record
for _, row in golden_record.iterrows():
    entity_type = row['entity_type']
    
    # Check if we have a dataframe for this entity type
    if entity_type in entity_dataframes:
        df = entity_dataframes[entity_type]
        
        # Find the matching row using original_id
        mask = df['identifier'] == row['original_id']
        
        # If a match is found, update the specified field with the varied value
        if mask.any():
            field_name = row['field_name']
            if field_name in df.columns:
                df.loc[mask, field_name] = row['varied_value']
                print(f"Updated {entity_type} - {row['original_id']} - {field_name}: {row['varied_value']}")
            else:
                print(f"Warning: Field {field_name} not found in {entity_type} dataframe")
        else:
            print(f"Warning: No row with identifier {row['original_id']} found in {entity_type} dataframe")
    else:
        print(f"Warning: No dataframe found for entity type {entity_type}")

Updated Address - 4e48ce3c-38a1-408d-b06f-0d4ac4092209 - country: Netherlands
Updated Address - 9cdeb3e6-0870-415c-afcd-81b5d24bace4 - postalCode: 64161 
Updated Address - 68cce369-339b-4ce8-8bb3-66b225a3fcce - country: Austria
Updated Address - d4e3fd71-e861-43fd-bca1-248409aaae6d - country: Austria
Updated Address - 3464a4d6-7f6e-45a6-80b3-5a929e0394e7 - city: Zeijerven
Updated Address - 5f3fa533-e28c-4c3c-ad82-5ea8dda9a02a - postalCode: 74582 
Updated Address - 50bea76b-e732-4e1f-a4b2-1ca69938b9e7 - postalCode: 5348TT
Updated Address - 1f206b2e-2e40-45cc-9bb9-967b19c87c07 - city: Bakhuizzen
Updated Address - 1a874912-066d-4941-9f7d-c765031c8276 - country: Austria
Updated Address - 0b9bcb7e-dba6-40f1-b5bb-cdb0bda31488 - postalCode: 95547 
Updated Address - 35d327e0-ab36-4a8f-a549-a9b73a30868e - city: Friestadt
Updated Address - 9c9da0d2-9087-4f0e-9206-d547d6a2c182 - postalCode: 5869 
Updated Address - a7b4913b-5624-4f64-87b9-2b789469d522 - postalCode: 2575PB
Updated Address - 45a1697

In [4]:
HealthcareOrganization_df

,identifier,healthcareOrganizationName,address,contactPoint
0,71545a13-7a1d-4006-8d72-3104f77383c1,Faulkner-Howard Zorg,c8a70639-eb11-47b3-a7a9-c3787c65c1e5,23a7711a-8133-4876-b7eb-dcd9e87a1613
1,6288e1a5-cc45-4821-98a6-416d1775336d,GSC Tervisekeskus,d3290a4c-b5d3-4b16-a619-4cb1d71037d1,b421eaeb-5340-47ca-baf3-897a3e70f16a
2,5f3f5638-3870-4a14-b490-b6081dfc8352,"Green, Jonhes and Shelton Tervisekeskus",9cdeb3e6-0870-415c-afcd-81b5d24bace4,09e469e6-ec62-42c8-a648-ee38e07405eb
3,870f084c-7244-4536-a85e-25b4b3969057,NAC Zorg,971c702d-5bf4-4c04-ac64-2b4c49b25ded,14aa451c-a69c-4b85-9432-f8db6a174c1c
4,ccc14d51-73f6-40d8-a9f4-1cc04653a560,LK Gesundheitszentrum,0cc36d8c-7786-4fe5-9675-ebf74fe30c9a,b9bdee2d-d663-449d-955e-18b1fa83ada4
5,fb5eb866-2640-411e-a9f2-c3c74505f4f6,WS Gesundheitszentrum,4745dd9e-2789-4389-9f32-77fd1d77ce40,f24dfdd8-5091-4bdc-8ef0-66d44279b14d
6,f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf,BI Tervisekeskus,15ace7a1-ceca-4ee3-90da-8a9516408169,8e751eb7-64d0-4913-991b-8adf0202861c
7,c2472fd6-03e9-4a02-8cea-2df00a66dc4e,Garcia PLC Gesundheitszentrum,d3b564b0-8be0-4c3e-9c94-938160c6b3ed,a36bcb01-67e9-4363-905c-053b25fdacbe
8,f164f9d8-4312-4ce2-9c21-51e17e56ac3d,Santana-Shaw Zorg,8147a8f4-5f0e-4320-b7f6-0e7f75f2bc20,ce1bb02a-cb4d-48d6-adb6-da351734a26c
9,586f1721-0785-48d7-b118-2d235bf80676,CHF Zorg,e38690e7-e27a-48e9-91c3-d1bcc6be6432,6d0c62c3-254b-47ae-9d0a-b994f20b575d


In [5]:
from faker import Faker

fake = Faker()

# Create a copy to avoid modifying original data
A_copy = Address_df.copy()
CP_copy = ContactPoint_df.copy()
P_copy = Person_df.copy()
HO_copy = HealthcareOrganization_df.copy()
SD_copy = ServiceDepartment_df.copy()
HP_copy = HealthcarePersonnel_df.copy()

# Save old values
old_valuesHO = {
    'identifier': HO_copy['identifier'].copy(),
    'address': HO_copy['address'].copy(),
    'contactPoint': HO_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field
HO_copy['identifier'] = HO_copy['identifier'].apply(lambda x: fake.uuid4())
HO_copy['address'] = HO_copy['address'].apply(lambda x: fake.uuid4())
HO_copy['contactPoint'] = HO_copy['contactPoint'].apply(lambda x: fake.uuid4())

# Create copies of dataframes
old_valuesSD = {
    'identifier': SD_copy['identifier'].copy(),
    'address': SD_copy['address'].copy(),
    'contactPoint': SD_copy['contactPoint'].copy()
}

# Generate new UUIDs for each field in ServiceDepartment_df
SD_copy['identifier'] = SD_copy['identifier'].apply(lambda x: fake.uuid4())
SD_copy['address'] = SD_copy['address'].apply(lambda x: fake.uuid4())
SD_copy['contactPoint'] = SD_copy['contactPoint'].apply(lambda x: fake.uuid4())

old_valuesHP = {
    'identifier': HP_copy['identifier'].copy()
}
# Generate new UUIDs for Healthcare Personnel identifiers
HP_copy['identifier'] = HP_copy['identifier'].apply(lambda x: fake.uuid4())

In [6]:
HO_copy

,identifier,healthcareOrganizationName,address,contactPoint
0,b243dc6f-11dc-472e-b759-ac99058c67e5,Faulkner-Howard Zorg,e5200bf2-d18d-4719-950f-bc9e3a36c440,6283f286-08b5-487e-b6d1-7b5af4997d82
1,9855b21b-482f-4b57-ae70-60ed9b0c7994,GSC Tervisekeskus,56f76566-aba0-4620-9d3e-f562231569a7,5c13a7b3-0ad6-4dba-b5cf-ab29c390625f
2,d440eaa0-6730-41e4-a335-ad0f48fc12c2,"Green, Jonhes and Shelton Tervisekeskus",146eba09-5b02-470a-8b12-b7f809f451aa,a2f7b5c4-3091-4c86-8350-c05b19e2b260
3,74234e2a-f5f7-4549-84d4-b3fb60f1643e,NAC Zorg,23deb92a-51cd-4d11-864f-4edfbda26d94,d0171850-095c-4648-ac27-3663a7d72443
4,2c8c0beb-9bce-49df-913b-964aa84dac35,LK Gesundheitszentrum,821cde87-558e-4641-9985-92a133823440,e78dac18-c75a-49d4-8a4b-09ede2ad078a
5,00626dbe-7ea1-473b-bfc3-ab58f03674f2,WS Gesundheitszentrum,5047f5fb-05e9-4d2a-9ce6-9704f933dff8,7378d658-4898-41fc-b33a-ae48c7a5bf59
6,e33e0318-9ca0-412a-9618-394618d63ced,BI Tervisekeskus,d17cc0c7-a658-42a5-bd1e-9a0abcd4a2d0,4604a1f9-6bd2-4490-b0f1-acb372a8104a
7,a16c836c-72ce-4aa1-9fdd-faf1d67bd1c8,Garcia PLC Gesundheitszentrum,dcdcf782-c2d5-4b09-b6a7-8464bacafa84,26d1d175-3968-4515-bd16-e856456e2bd9
8,2d25e454-dcd5-44a2-9fc2-66401ecbd8fc,Santana-Shaw Zorg,e286fc8f-b0c6-4b57-ad51-3dec74724453,e6689486-6233-460d-96bc-8660178d4261
9,42048a1e-a15c-4bd3-b572-ff992c460449,CHF Zorg,ae88a7a8-1c11-437d-8bf9-35c5b575b722,61823e7b-9614-465a-a5d7-9ed8f2cbc9ea


In [7]:
# Create a mapping of old to new UUIDs
HO_uuid_mapping = {
    old_valuesHO['identifier'].iloc[i]: HO_copy['identifier'].iloc[i]
    for i in range(len(HO_copy))
}

# Update references in other dataframes (using the copies)
# Update institution references in Healthcare_Personnel_df copy
HP_copy['institution'] = HP_copy['institution'].map(HO_uuid_mapping).fillna(HP_copy['institution'])

# Update isPartOf references in ServiceDepartment_df copy
SD_copy['isPartOf'] = SD_copy['isPartOf'].map(HO_uuid_mapping).fillna(SD_copy['isPartOf'])

# Update contact point references
ho_contact_mapping = {
    old_valuesHO['contactPoint'].iloc[i]: HO_copy['contactPoint'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in ContactPoint_df copy where it matches old contact points
CP_copy['identifier'] = CP_copy['identifier'].map(ho_contact_mapping).fillna(CP_copy['identifier'])

# Update address references
ho_address_mapping = {
    old_valuesHO['address'].iloc[i]: HO_copy['address'].iloc[i]
    for i in range(len(HO_copy))
}

# Update identifier in Address_df copy where it matches old addresses
A_copy['identifier'] = A_copy['identifier'].map(ho_address_mapping).fillna(A_copy['identifier'])

# Create mapping for ServiceDepartment
sd_uuid_mapping = {
    old_valuesSD['identifier'].iloc[i]: SD_copy['identifier'].iloc[i]
    for i in range(len(SD_copy))
}

# Create address mapping for ServiceDepartment
sd_address_mapping = {
    old_valuesSD['address'].iloc[i]: SD_copy['address'].iloc[i]
    for i in range(len(SD_copy))
}

# Create contact point mapping for ServiceDepartment
sd_contact_mapping = {
    old_valuesSD['contactPoint'].iloc[i]: SD_copy['contactPoint'].iloc[i]
    for i in range(len(SD_copy))
}

# Update references in Address_df copy for ServiceDepartment
A_copy['identifier'] = A_copy['identifier'].map(sd_address_mapping).fillna(A_copy['identifier'])

# Update references in ContactPoint_df copy for ServiceDepartment
CP_copy['identifier'] = CP_copy['identifier'].map(sd_contact_mapping).fillna(CP_copy['identifier'])

# Create mapping for Personnel
hp_uuid_mapping = {
    old_valuesHP['identifier'].iloc[i]: HP_copy['identifier'].iloc[i]
    for i in range(len(HP_copy))
}

# Update references in Person_df copy
P_copy['identifier'] = P_copy['identifier'].map(hp_uuid_mapping).fillna(P_copy['identifier'])

In [8]:
old_valuesHO

{'identifier': 0     71545a13-7a1d-4006-8d72-3104f77383c1
 1     6288e1a5-cc45-4821-98a6-416d1775336d
 2     5f3f5638-3870-4a14-b490-b6081dfc8352
 3     870f084c-7244-4536-a85e-25b4b3969057
 4     ccc14d51-73f6-40d8-a9f4-1cc04653a560
 5     fb5eb866-2640-411e-a9f2-c3c74505f4f6
 6     f2ad985f-ff3e-4ba1-8ac7-28b4a41865bf
 7     c2472fd6-03e9-4a02-8cea-2df00a66dc4e
 8     f164f9d8-4312-4ce2-9c21-51e17e56ac3d
 9     586f1721-0785-48d7-b118-2d235bf80676
 10    4ec985ff-94b2-4b9d-8881-9f421a42b629
 11    788c161e-f3cc-4d8a-8b16-78e45f20f406
 12    48622a67-4a29-4067-9c66-a27b6a325333
 13    862f78e1-25c2-4b6a-b575-bc6eeee84bcb
 14    f46860a2-a604-45af-b4db-1c1eaa66b464
 15    0b5aafef-85a1-482a-8761-585b4168705a
 16    3fee2fc7-2e2d-4fdf-b57b-8a920a12f3b3
 17    1e454241-45c1-40a8-a52b-a0ce627b585f
 18    839137a6-a977-4b71-9e6d-2d7e9f312acb
 19    f1c58f44-7e08-4c1b-8da6-a326451437d6
 20    909b471c-0d1f-4a7f-90fc-e92295be263b
 21    ce8c037e-e51c-4cdb-8247-34dfdd1f8e30
 22    53f3c158-42

In [9]:
import os

# Create the new directory if it doesn't exist
if not os.path.exists('testdata_dup'):
    os.makedirs('testdata_dup')

# Save each dataframe to CSV in the new directory
A_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Address.csv', index=False)
CP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ContactPoint.csv', index=False)
HO_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcareOrganization.csv', index=False)
HP_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/HealthcarePersonnel.csv', index=False)
P_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/Person.csv', index=False)
SD_copy.to_csv('Data_source/Sample_35_train/train_data_dupe/ServiceDepartment.csv', index=False)

In [10]:
import pandas as pd
import os

# Define paths
original_data_path = 'Data_source/Sample_35_train/train_data'
duplicate_data_path = 'Data_source/Sample_35_train/train_data_dupe'
golden_standard_path = 'train_golden_standard_duplicates.csv'
output_path = 'train_golden_standard_duplicates_final.csv'

# Read golden standard file
golden_df = pd.read_csv(golden_standard_path, encoding='latin1')

# Dictionary to store matching field values between original and duplicate datasets
field_value_to_id_mapping = {}

# Process each entity type
for _, row in golden_df.iterrows():
    entity_type = row['entity_type']
    field_name = row['field_name']
    varied_value = row['varied_value']
    
    # Define file paths based on entity type
    if entity_type == 'Address':
        original_file = 'Address.csv'
        duplicate_file = 'Address.csv'
    elif entity_type == 'ContactPoint':
        original_file = 'ContactPoint.csv'
        duplicate_file = 'ContactPoint.csv'
    elif entity_type == 'HealthcareOrganization':
        original_file = 'HealthcareOrganization.csv'
        duplicate_file = 'HealthcareOrganization.csv'
    elif entity_type == 'HealthcarePersonnel':
        original_file = 'HealthcarePersonnel.csv'
        duplicate_file = 'HealthcarePersonnel.csv'
    elif entity_type == 'Person':
        original_file = 'Person.csv'
        duplicate_file = 'Person.csv'
    elif entity_type == 'ServiceDepartment':
        original_file = 'ServiceDepartment.csv'
        duplicate_file = 'ServiceDepartment.csv'
    else:
        continue
    
    # Read duplicate dataset
    duplicate_df = pd.read_csv(os.path.join(duplicate_data_path, duplicate_file))
    
    # Look for records in the duplicate dataset that have the varied value
    # in the specified field
    if field_name in duplicate_df.columns:
        matching_rows = duplicate_df[duplicate_df[field_name] == varied_value]
        
        if not matching_rows.empty:
            # Get the identifier from the first matching row
            new_duplicate_id = matching_rows.iloc[0]['identifier']
            
            # Store the mapping for this specific row
            key = (entity_type, field_name, varied_value)
            field_value_to_id_mapping[key] = new_duplicate_id

# Create a new dataframe for the updated golden standard
updated_golden_df = golden_df.copy()

# Update the duplicate_id column based on our mappings
for i, row in updated_golden_df.iterrows():
    key = (row['entity_type'], row['field_name'], row['varied_value'])
    if key in field_value_to_id_mapping:
        updated_golden_df.at[i, 'duplicate_id'] = field_value_to_id_mapping[key]

# Save the updated golden standard file
updated_golden_df.to_csv(output_path, index=False, encoding='utf-8')

print(f"Updated golden standard file saved to {output_path}")
print(f"Total field-value mappings created: {len(field_value_to_id_mapping)}")
print(f"Duplicate IDs updated: {sum(updated_golden_df['duplicate_id'] != golden_df['duplicate_id'])}")

Updated golden standard file saved to train_golden_standard_duplicates_final.csv
Total field-value mappings created: 1917
Duplicate IDs updated: 2228
